Copy this Colab notebook into your own Drive and solve it locally.

# Lab 06 - TLS Attacks - BEAST

In this lab, we will implement the BEAST (Browser Exploit Against SSL/TLS) attack that targets TLS/SSL versions up until TLS 1.0.

SSLv3/TLS1.0 are protocols to encrypt/decrypt and secure your data. In our case, they both use the CBC cipher mode chaining. The plaintext is divided into block regarding the encryption algorithm (AES, DES, 3DES) and the length is a multiple of 8 or 16. If the plaintext does not fill the length, a padding is added at the end to complete the missing space. In CBC mode, each block of plaintext is XORed with the previous ciphertext block before being encrypted. This way, each ciphertext block depends on all plaintext blocks processed up to that point.

```
C[i] = Ek(P[i] XOR C[i-1]), C[0] = IV

P[i] = Dk(C[i]) XOR C[i-1], P[0] = IV
```

To make each message unique, an initialization vector must be used in the first block. When encrypting with CBC mode, the Initialization Vector (IV) is:

- Random
- Unpredictable
- Not secret

<img src='https://ocw.cs.pub.ro/courses/_media/ac/laboratoare/601px-cbc_encryption.svg.png'>

<img src='https://ocw.cs.pub.ro/courses/_media/ac/laboratoare/601px-cbc_decryption.svg.png'>

## BEAST Attack

 In TLS1.0 and SSLv3 the first IV of the request is random, fine. But to gain some time and not generate a new random IV every time, the implementation of TLS1.0 and SSLv3 used the last block of the previous cipher text has an IV. In other words, the IV is now guessable. We will assume the length of each block will be 16 (AES) and the attacker can set up a MITM to retrieve all the ciphertext. Additionally, the attacker can inject malicious code in the victim's browser and can use a socket to send data in the name of the client. This is called a [Chosen plaintext attack](https://en.wikipedia.org/wiki/Chosen-plaintext_attack) and can be seen below. Original description: [BEAST paper 2011](https://nerdoholic.org/uploads/dergln/beast_part2/ssl_jun21.pdf).

 <img src='https://ocw.cs.pub.ro/courses/_media/ac/laboratoare/beast_attack.png'>

In this attack we assume the client is using a secret COOKIE to authenticate to the server and the attacker would like to obtain this COOKIE. Furthermore, we assume that the attacker can force the client (via the malicious script) to send an encryption of any message followed by the secret COOKIE (this would be a response to a server request).

Let's say that we want to find the first character of the COOKIE, we will instruct the victim to encrypt the following message: `'B'*15 + COOKIE`. Remember that the block length is 16 bytes, therefore the first block of the message will be `'BBBBBBBBBBBBBBB?'` (let's call it `m1`), where `?` represents the unknown byte.

Because this is the first request, the IV will be random. Observing the ciphertext, we can extract two key pieces of information: `Ek(m1 XOR IV)` and `F[-16:]` will be the IV for the second request. Here `F` is the encryption of this first request.

For the second request, we know what the IV will be (due to how CBC works). Let's send the same message as in the first request - we will get `Ek(m1 XOR F[-16:0])` as the first block. Last block of the ciphertext (`S[-16:]`) will be the next IV. Here, `S` is the encryption of this second request.

For the third request, we will craft a special block `P = m_guess XOR F[-16:] XOR S[-16:0]`, where `m_guess` is `'B'*15` + our guess for the first character of the cookie. Try to find what the first block of the third ciphertext should look like. Use the technique to find the entire secret cookie.

### Useful functions and setup

In [1]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.9 MB/s eta 0:00:00


In [2]:
from Crypto.Cipher import AES
from Crypto import Random

SECRET_COOKIE = b"ID=3ef729ccf0cc5" # Secret value, don't access it directly!
BLOCK_LENGTH = 16

last_iv = None

def pad(s):
    return s + (16 - len(s) % 16) * chr(16 - len(s) % 16).encode()

def unpad(s):
    return s[:-ord(s[len(s)-1:])]

def xor_strings(xs, ys, zs):
    result = bytearray()
    for x, y, z in zip(xs, ys, zs):
        result.append(x ^ y ^ z)
    return bytes(result)

def xor_block(vector_init, previous_cipher, p_guess):
    xored = xor_strings(vector_init, previous_cipher, p_guess)
    return xored

def split_len(seq, length):
    return [seq[i:i+length] for i in range(0, len(seq), length)]

def encrypt(msg, iv_p=None):
    raw = pad(msg + SECRET_COOKIE)
    if iv_p is None:
        iv = Random.new().read(AES.block_size)
    else:
        iv = iv_p
    global key
    key = Random.new().read(AES.block_size)
    cipher = AES.new(b'V38lKILOJmtpQMHp', AES.MODE_CBC, iv)
    return cipher.encrypt(raw)

def send_request(msg):
    global last_iv
    enc = encrypt(msg, last_iv)
    last_iv = enc[-BLOCK_LENGTH:]
    return enc

## Task - BEAST PoC implementation

Note: do not access `SECRET_COOKIE` directly.

In [3]:
def run_three_request():
    secret = []

    # the part of the request the atacker knows, can be null
    known_so_far = b""

    # retrieve all the bytes of the cookie (in total 16 bytes, all fit in one block)
    for t in range(len(SECRET_COOKIE)):
        padding = 16 - len(known_so_far) - 1
        padding_bytes = b'B' * padding

        for i in range(0,256):

            guess = bytearray(known_so_far)
            guess.append(i)
            guess=bytes(guess)

            m_guess = padding_bytes + guess

            # TODO send first request
            C1 = send_request(padding_bytes)
            IV2 = C1[-BLOCK_LENGTH:]

            # TODO send second request
            C2 = send_request(padding_bytes)
            C2_target_block = C2[:BLOCK_LENGTH]
            IV3 = C2[-BLOCK_LENGTH:]

            # TODO craft and send third request
            P_crafted = xor_block(IV3, IV2, m_guess)
            C3 = send_request(P_crafted)
            C3_result_block = C3[:BLOCK_LENGTH]

            # TODO check if the guess is correct
            if C3_result_block == C2_target_block:
                known_so_far = bytearray(known_so_far)
                known_so_far.append(i)
                break
            elif i == 255:
                print("Unable to find the char...")
                return known_so_far
    return known_so_far


# the attacker doesn't know the cookie
secret = bytes(run_three_request())

print(secret)

b'ID=3ef729ccf0cc5'
